# Top commodities in trade between partners

This notebook will collect data on the top commodities in trade between partners for a given period.

Partners can be a single country or a group of countries



## Setup 

See [here](0-comtrade-setup-first.ipynb) for instructions on how to obtain an API Key.
Without APIKEY results are limited to a maximum of 500 rows.


In [1]:
import comtradetools as comtradetools

comtradetools.setup()
APIKEY = comtradetools.get_api_key()
comtradetools.init(APIKEY, force_init=False)

INFO:root:Loading HS codes from support/harmonized-system.csv


## Top commodities in trade between partners

Obtains main commodities in trade between partners for a given period, using 2 digit HS codes.

For the meaning of codes see: https://www.wcoomd.org/en/topics/nomenclature/instrument-and-tools/hs-nomenclature-2022-edition/hs-nomenclature-2022-edition.aspx


### TODO

- [ ] Permitir escolher os reporter e partners
- [ ] compatibilizar com o formato de report da AICEP https://myaicep.portugalexporta.com/mercados-internacionais/cn/china?setorProduto=-1
- [ ] há problemas de duplicação de linhas nas listagem de detalhes de commodities, provavlmente por custom code, ou mot code.
  

Define:
* `reporter` - country or countries providing the data
* `partner` - country or countries trading with the reporter
* `years_of_interest` - year or period for which data is requested
* `rank_filter` - number of top commodities to be analyzed
* `flowCode` M for imports, X for exports, M,X for both

In [2]:
import comtradetools as comtradetools

reporterDesc = 'China'
reporterCode = comtradetools.encode_country(reporterDesc)
partnerDesc = 'PLP'
partnerCode = comtradetools.m49_plp_list
year_start = 2003
year_end = 2023
years_of_interest = comtradetools.year_range(year_start,year_end+1)
rank_filter = 5  # número de importações mais relevantes
flowCode = 'M,X'


In [3]:
import comtradetools as comtrade
import pandas as pd



pco_cols = ['reporterDesc','partnerDesc','refYear','pyf_rank','cmdCode','cmdDesc',
            'flowCode','primaryValue','pyf_sum','pyf_perc']

periods = comtrade.split_period(years_of_interest,12)
df = pd.DataFrame()
for period in periods:
    print(f"Fetching data for period {period}")

    temp = comtrade.getFinalData(APIKEY,
                                typeCode="C",# C for commodities, S for Services
                                freqCode="A",# (freqCode) A for annual and M for monthly
                                flowCode=flowCode,
                                cmdCode="AG2",
                                reporterCode=reporterCode,
                                partnerCode=partnerCode,
                                period=period,
                                partner2Code=0,
                                clCode="HS",
                                customsCode='C00',
                                motCode=0,
                                includeDesc=True,
                                cache=True,
                                use_alternative=False
                                )
    df = pd.concat([df,temp], ignore_index=True)

# Subtotal by partner, year (not flow?)
pco = df.sort_values(['partnerDesc','refYear','primaryValue'], ascending=[True,True,False])
# Determine rank of each partner in import and export
pco['pyf_rank'] = pco.groupby(['partnerDesc','refYear','flowCode'])["primaryValue"].rank(method="dense", ascending=False)
pco['pyf_sum'] = pco.groupby(['partnerDesc','refYear','flowCode'])["primaryValue"].transform('sum')
pco['pyf_perc'] = pco['primaryValue'] / pco['pyf_sum']
# convert rank column to int
pco['pyf_rank'] = pco['pyf_rank'].astype(int)

# filter top partners
pco_tops = pco[pco['pyf_rank'] <= rank_filter]
cmdCodes_tops = pco_tops['cmdCode'].unique()
print(f"Product categories involved in exchanges with top partners:\n {cmdCodes_tops}")




INFO:root:Calling getFinalData for period 2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014
INFO:root:Using cached results for period 2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014
INFO:root:Calling getFinalData for period 2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
INFO:root:Using cached results for period 2015,2016,2017,2018,2019,2020,2021,2022,2023,2024


Fetching data for period 2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014
Fetching data for period 2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
Product categories involved in exchanges with top partners:
 ['27' '85' '64' '87' '52' '84' '25' '71' '39' '20' '90' '40' '73' '44'
 '03' '72' '74' '76' '94' '68' '62' '22' '01' '26' '63' '12' '47' '15'
 '29' '54' '41' '17' '02' '89' '38' '10' '95' '70' '34' '60' '61' '09'
 '16' '69' '99' '92' '49' '82' '48' '65' '97' '33' '30' '56' '08' '04'
 '21' '55' '96' '28' '31' '86' '45' '42' '83' '06' '07' '32' '14' '91'
 '78' '46']


In [4]:
# show
pd.options.display.max_colwidth=100
pd.options.display.max_rows=1600
pd.options.display.float_format = '{:,.2f}'.format

pco_tops_sorted = pco_tops[pco_cols].set_index(['reporterDesc','partnerDesc','flowCode','refYear','pyf_rank']).sort_index()
pco_tops_sorted.head(10).style.format(comtrade.make_format(list(pco_tops_sorted.columns)))


## Output to Excel file

In [5]:
# Prepare file name for Excel output

real_period = pco_tops_sorted.index.get_level_values('refYear').unique()
start_year=min(real_period)
end_year=max(real_period)
period=f"{start_year}-{end_year}"

filename_note=f"{period}_{flowCode.replace(',','_')}"  # change to append to filename
excel_file_name = f"./reports/{reporterDesc}_{partnerDesc}_tops_{filename_note}.xlsx"
excel_file = pd.ExcelWriter(excel_file_name)
# Save data
sheet_name = "comtrade"
pco_tops_sorted.reset_index().to_excel(excel_file, sheet_name=sheet_name, index=True, startrow=2)
# Adjust column width
comtrade.excel_col_autowidth(pco_tops_sorted, excel_file,sheet_name)

# format column primaryValue as currency
comtrade.excel_format_currency(pco_tops_sorted, excel_file,sheet_name, columns=['primaryValue'],width=20)

# Write title in first row TBD
excel_file_title = f"Commodities {reporterDesc}-{partnerDesc}, top {rank_filter} (M=Import, X=Export), {period} USD"
print(excel_file_title)
excel_file.close()
print("Saved to:",excel_file_name)


Commodities China-PLP, top 5 (M=Import, X=Export), 2003-2022 USD
Saved to: ./reports/China_PLP_tops_2003-2022_M_X.xlsx


## Analysis of variations in top commodities flows over time

The next cell provides a function that will analyze the variations in the top commodities flows over time.

For each combination of Reporter, Partner Year and Flow, the function will produce a list of the top commodities in trade between partners for a given period, using 2 digit HS codes.
```
China Brazil M
2003   12,26,72 | 12=1,683,531,130.00 (100%) | 26=1,369,127,929.00 (100%) | 72=745,347,034.00 (100%) | 
        -> 26 Ores, slag and ash
        -> 12 Oil seeds and oleaginous fruits; miscellaneous grains, seeds and fruit, industrial or medicinal plants; straw and fodder
        -> 72 Iron and steel
2015 ! 12,26,27 | 12=16,887,916,146.00  (1003%) | 26=12,653,641,892.00  (924%) | 27=5,297,996,353.00  (new:27) | 
        -> 27 Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes
2016 = 12,26,27 | 12=15,551,903,256.00 (92%) | 26=13,339,552,748.00 (105%) | 27=6,041,262,180.00 (114%) | 
2017 = 12,26,27 | 12=20,916,408,256.00 (134%) | 26=18,261,973,405.00 (137%) | 27=9,189,149,994.00 (152%) | 
2018 = 12,26,27 | 12=28,843,062,797.00 (138%) | 26=19,307,536,897.00 (106%) | 27=16,484,023,407.00 (179%) | 
12 Oil seeds and oleaginous fruits; miscellaneous grains, seeds and fruit, industrial or medicinal plants; straw and fodder
26 Ores, slag and ash
27 Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes
72 Iron and steel
```
Each year is followed by a character that indicates the variation in the top commodities from the previous year:
* ! : change in one of the top commodities
* = : same top commodities as the previous year, same relative importance
* ≈ : same top commodities as the previous year, different relative importance

Change `consider_first` to set the number of top commodities to be examined.

In [6]:
consider_first = 3
for reporter in pco_tops_sorted.index.get_level_values(0).unique():
    for partner in pco_tops_sorted.loc[(reporter,)].index.get_level_values(0).unique(): 

        for flow in pco_tops_sorted.loc[((reporter,partner,))].index.get_level_values(0).unique():
            print(reporter,partner, flow)
            print("Percentages show change of commodity value compared to previous year")
            previous_pattern = []
            all_commodities = set() # all commodities for this partner for this flow
            all_commodities_desc = set() # all commodities for this partner for this flow descriptions
            previous_years_values = dict()
            for year in pco_tops_sorted.loc[(reporter,partner,flow,)].index.get_level_values(0).unique():
                print(year, end=' ')
                top_codes = pco_tops_sorted.loc[(reporter,partner,flow,year)]['cmdCode'].astype(str).values.flatten().tolist()[0:consider_first]
                top_codes_desc = pco_tops_sorted.loc[(reporter,partner,flow,year)]['cmdDesc'].astype(str).values.flatten().tolist()[0:consider_first]
                top_primary_value = pco_tops_sorted.loc[(reporter,partner,flow,year)]['primaryValue'].values.flatten().tolist()[0:consider_first]
                top_table = {code: {"desc":desc, "value":value, "year": year} for code,desc,value in zip(top_codes,top_codes_desc,top_primary_value)}
                codes_desc = {f"{code} {desc}" for code,desc in zip(top_codes,top_codes_desc)}
                # make set of tuples top5_codes and top5_codes_desc
                new_commodities = set(top_codes).difference(all_commodities)
                all_commodities = all_commodities.union(set(top_codes))
                all_commodities_desc = all_commodities_desc.union(codes_desc)
                pattern = ','.join(top_codes)
                if len(previous_pattern) == 0:
                    previous_pattern = top_codes
                    # update previous_years_values with top5_table
                    previous_years_values.update(top_table)
                    print(' ', pattern, end=' | ')
                    for cmd in top_codes:
                        print(f"{cmd}={top_table[cmd]['value']:,.2f} ({top_table[cmd]['value'] / previous_years_values[cmd]['value']:.0%})", end=' | ')
                    print()
                    for cmd in new_commodities:
                        print( " "*13+"  ->", cmd,top_table[cmd]['desc'])
                elif set(top_codes) == set(previous_pattern):
                    if top_codes == previous_pattern:
                        print('=', pattern, end=' | ')
                    else:
                        print('≈', pattern, end=' | ')
                    for cmd in top_codes:
                        print(f"{cmd}={top_table[cmd]['value']:,.2f} ({top_table[cmd]['value'] / previous_years_values[cmd]['value']:.0%})", end=' | ')
                    print()
                else:
                    print('!', pattern, end=' | ')
                    for cmd in top_codes:
                        print(f"{cmd}={top_table[cmd]['value']:,.2f} ", end=' ')
                        if cmd in previous_years_values.keys():
                            print(f"({top_table[cmd]['value'] / previous_years_values[cmd]['value']:.0%})", end=' | ')
                        else:
                            print(f"(new:{cmd})", end=' | ')
                    print()
                    if len(new_commodities) > 0:
                        for cmd in new_commodities:
                            print( " "*13+"  ->", cmd, top_table[cmd]['desc'])
                       
                previous_pattern = top_codes
                previous_years_values.update(top_table)
            for cmd in sorted(all_commodities_desc):
                print( "    ",cmd)
            print(80*"-")



China Angola M
Percentages show change of commodity value compared to previous year
2003   27,25,71 | 27=2,205,654,993.00 (100%) | 25=232,059.00 (100%) | 71=47,397.00 (100%) | 
               -> 25 Salt; sulphur; earths, stone; plastering materials, lime and cement
               -> 27 Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes
               -> 71 Natural, cultured pearls; precious, semi-precious stones; precious metals, metals clad with precious metal, and articles thereof; imitation jewellery; coin
2004 ≈ 27,71,25 | 27=4,716,807,801.00 (214%) | 71=430,188.00 (908%) | 25=92,383.00 (40%) | 
2005 ! 27,71,73 | 27=6,575,939,442.00  (139%) | 71=5,645,659.00  (1312%) | 73=120,907.00  (new:73) | 
               -> 73 Iron or steel articles
2006 ! 27,71,25 | 27=10,930,839,396.00  (166%) | 71=2,162,576.00  (38%) | 25=128,801.00  (139%) | 
2007 ! 27,71,44 | 27=12,879,534,443.00  (118%) | 71=8,619,506.00  (399%) | 44=338,307.00  (new:44)

## Detail of the top commodities

Examine the detail fo the top level commodities.

Top level commodities are expressed in two digit codes.
For each two digit code in the top commodities we fetch all the lowerlevel codes to examine the details.

Two steps: 
* Get the two character codes of the most relevant commodities in reporter partner trade.
* Fetch all the subcodes of each two character codes

To decode the detailed codes see ver: https://www.wcoomd.org/en/topics/nomenclature/instrument-and-tools/hs-nomenclature-2022-edition/hs-nomenclature-2022-edition.aspx

In [59]:
import comtradetools
import time
import pandas as pd


consider_first = 3
use_cache = True
comtradetools.CALLS_PER_PERIOD = 1
comtradetools.PERIOD_SECONDS = 5

pco_cols = ['reporterDesc','partnerDesc','refYear','rank','cmdDesc',
            'flowCode','primaryValueFormated']

pco_cols_detail = ['reporterDesc','partnerDesc','refYear','cmdCodeAG2','motDesc','cmdCode','cmdDesc',
            'flowCode','primaryValue', 'isAggregate']

detail_commodities = pd.DataFrame()
for reporter in pco_tops_sorted.index.get_level_values(0).unique():
    for partner in pco_tops_sorted.loc[(reporter,)].index.get_level_values(0).unique(): 

        for flow in pco_tops_sorted.loc[((reporter,partner,))].index.get_level_values(0).unique():
            print(reporter,partner, flow)
            years = sorted(pco_tops_sorted.loc[(reporter,partner,flow,)].index.get_level_values(0).unique())
            for year in [years[0],years[-1]]:
                print(year, end=' ')
                top_codes = pco_tops_sorted.loc[(reporter,partner,flow,year)]['cmdCode'].astype(str).values.flatten().tolist()[0:consider_first]
                # get the top level codes from the previous step
                top_codes_desc = pco_tops_sorted.loc[(reporter,partner,flow,year)]['cmdDesc'].astype(str).values.flatten().tolist()[0:consider_first]
                # get all the detailed codes from the reporter / partner / flow / year
                df = comtradetools.getFinalData(
                    APIKEY,
                    typeCode="C",# C for commodities, S for Services
                    freqCode="A",# (freqCode) A for annual and M for monthly
                    flowCode=flow,
                    clCode="HS",
                    #cmdCode=country_cmd_codes, # sometimes the detail commodity codes generates errors
                    cmdCode=None, # This gives all the commodities for the country and year 
                    reporterCode=comtradetools.encode_country(reporter),
                    partnerCode=comtradetools.encode_country(partner),
                    period=str(year),
                    partner2Code=0,
                    motCode=0,
                    customsCode='C00',
                    includeDesc=True,
                    cache=use_cache,
                    use_alternative=False
                    )
                # create a column with the first 2 digits of the commodity code
                df['cmdCodeAG2'] = df.cmdCode.str[0:2]
                # filter the detail commodity codes by the top commodity codes, disregard the rest
                df = df[df['cmdCodeAG2'].isin(top_codes)]
                # show the top level code and the detail
                print(df['cmdCodeAG2'].unique(),df[df['isLeaf']]['cmdCode'].unique())
                detail_commodities = pd.concat([detail_commodities,df], ignore_index=True)

INFO:root:Calling getFinalData for period 2003
INFO:root:Using cached results for period 2003
INFO:root:Calling getFinalData for period 2022
INFO:root:Using cached results for period 2022
INFO:root:Calling getFinalData for period 2003
INFO:root:Using cached results for period 2003
INFO:root:Calling getFinalData for period 2022
INFO:root:Using cached results for period 2022
INFO:root:Calling getFinalData for period 2003
INFO:root:Using cached results for period 2003
INFO:root:Calling getFinalData for period 2022
INFO:root:Using cached results for period 2022
INFO:root:Calling getFinalData for period 2003
INFO:root:Using cached results for period 2003
INFO:root:Calling getFinalData for period 2022
INFO:root:Using cached results for period 2022
INFO:root:Calling getFinalData for period 2003
INFO:root:Using cached results for period 2003
INFO:root:Calling getFinalData for period 2022
INFO:root:Using cached results for period 2022
INFO:root:Calling getFinalData for period 2003
INFO:root:Usi

China Angola M
2003 ['25' '27' '71'] ['251611' '270900' '710231']
2022 ['25' '27' '74'] ['250610' '250620' '251511' '251512' '251611' '251612' '253010' '253090'
 '270900' '271112' '271113' '271500' '740312' '740313' '740319' '740321'
 '740400' '740819']
China Angola X
2003 ['64' '85' '87'] ['640191' '640192' '640219' '640220' '640299' '640340' '640359' '640399'
 '640419' '640520' '640590' '640610' '640620' '850132' '850151' '850152'
 '850161' '850211' '850212' '850220' '850300' '850431' '850440' '850490'
 '850610' '850680' '850710' '850720' '850790' '850910' '850940' '850980'
 '850990' '851220' '851230' '851310' '851511' '851580' '851610' '851629'
 '851631' '851640' '851650' '851660' '851671' '851672' '851679' '851690'
 '851711' '851719' '851730' '851790' '851810' '851821' '851822' '851830'
 '851840' '851850' '851890' '852190' '852290' '852311' '852313' '852520'
 '852713' '852731' '852732' '852812' '852813' '852830' '852910' '852990'
 '853090' '853110' '853180' '853222' '853310' '85354

INFO:root:Using cached results for period 2022
INFO:root:Calling getFinalData for period 2003
INFO:root:Using cached results for period 2003
INFO:root:Calling getFinalData for period 2022
INFO:root:Using cached results for period 2022
INFO:root:Calling getFinalData for period 2003
INFO:root:Using cached results for period 2003
INFO:root:Calling getFinalData for period 2022
INFO:root:Using cached results for period 2022
INFO:root:Calling getFinalData for period 2004
INFO:root:Using cached results for period 2004
INFO:root:Calling getFinalData for period 2022
INFO:root:Using cached results for period 2022
INFO:root:Calling getFinalData for period 2003
INFO:root:Using cached results for period 2003
INFO:root:Calling getFinalData for period 2022
INFO:root:Using cached results for period 2022
INFO:root:Calling getFinalData for period 2003
INFO:root:Using cached results for period 2003
INFO:root:Calling getFinalData for period 2022
INFO:root:Using cached results for period 2022
INFO:root:Cal

['16' '69' '84'] ['160414' '160415' '160419' '160420' '690410' '690490' '690510' '690721'
 '690722' '690723' '691010' '691090' '691110' '691190' '691310' '691410'
 '691490' '840290' '840790' '840810' '840991' '840999' '841231' '841319'
 '841320' '841330' '841350' '841360' '841370' '841381' '841391' '841410'
 '841420' '841430' '841440' '841451' '841459' '841460' '841480' '841490'
 '841510' '841581' '841583' '841590' '841810' '841821' '841830' '841840'
 '841850' '841861' '841869' '841899' '841911' '841919' '841920' '841939'
 '841981' '841989' '841990' '842121' '842123' '842129' '842131' '842139'
 '842199' '842211' '842230' '842240' '842310' '842389' '842390' '842410'
 '842420' '842482' '842489' '842542' '842710' '842720' '842790' '842820'
 '842833' '842839' '842840' '842890' '842951' '842952' '843061' '843120'
 '843149' '843210' '843229' '843510' '843621' '843780' '843810' '843850'
 '843860' '843880' '844110' '844180' '844331' '844332' '844339' '844399'
 '845011' '845012' '845019' '84502

INFO:root:Calling getFinalData for period 2022
INFO:root:Using cached results for period 2022
INFO:root:Calling getFinalData for period 2003
INFO:root:Using cached results for period 2003
INFO:root:Calling getFinalData for period 2022
INFO:root:Using cached results for period 2022
INFO:root:Calling getFinalData for period 2003
INFO:root:Using cached results for period 2003
INFO:root:Calling getFinalData for period 2022
INFO:root:Using cached results for period 2022
INFO:root:Calling getFinalData for period 2003
INFO:root:Using cached results for period 2003
INFO:root:Calling getFinalData for period 2022
INFO:root:Using cached results for period 2022
INFO:root:Calling getFinalData for period 2003
INFO:root:Using cached results for period 2003
INFO:root:Calling getFinalData for period 2022
INFO:root:Using cached results for period 2022
INFO:root:Calling getFinalData for period 2003
INFO:root:Using cached results for period 2003
INFO:root:Calling getFinalData for period 2022
INFO:root:Usi

['61' '85' '87'] ['610120' '610130' '610190' '610210' '610220' '610230' '610290' '610310'
 '610322' '610323' '610331' '610332' '610333' '610339' '610341' '610342'
 '610343' '610349' '610422' '610423' '610429' '610431' '610432' '610433'
 '610439' '610441' '610442' '610443' '610444' '610449' '610451' '610452'
 '610453' '610459' '610461' '610462' '610463' '610469' '610510' '610520'
 '610590' '610610' '610620' '610690' '610711' '610712' '610719' '610721'
 '610722' '610729' '610791' '610799' '610811' '610819' '610821' '610822'
 '610829' '610831' '610832' '610839' '610891' '610892' '610910' '610990'
 '611011' '611012' '611019' '611020' '611030' '611090' '611120' '611130'
 '611190' '611211' '611212' '611219' '611231' '611239' '611241' '611249'
 '611300' '611420' '611430' '611490' '611510' '611521' '611522' '611529'
 '611530' '611594' '611595' '611596' '611599' '611610' '611691' '611692'
 '611693' '611710' '611780' '611790' '850110' '850131' '850133' '850151'
 '850152' '850153' '850231' '85030

In [9]:
detail_commodities.columns

Index(['typeCode', 'freqCode', 'refPeriodId', 'refYear', 'refMonth', 'period',
       'reporterCode', 'reporterISO', 'reporterDesc', 'flowCode', 'flowDesc',
       'partnerCode', 'partnerISO', 'partnerDesc', 'partner2Code',
       'partner2ISO', 'partner2Desc', 'classificationCode',
       'classificationSearchCode', 'isOriginalClassification', 'cmdCode',
       'cmdDesc', 'aggrLevel', 'isLeaf', 'customsCode', 'customsDesc',
       'mosCode', 'motCode', 'motDesc', 'qtyUnitCode', 'qtyUnitAbbr', 'qty',
       'isQtyEstimated', 'altQtyUnitCode', 'altQtyUnitAbbr', 'altQty',
       'isAltQtyEstimated', 'netWgt', 'isNetWgtEstimated', 'grossWgt',
       'isGrossWgtEstimated', 'cifvalue', 'fobvalue', 'primaryValue',
       'legacyEstimationFlag', 'isReported', 'isAggregate', 'cmdCodeAG2'],
      dtype='object')

In [15]:
import comtradetools
# Prepare file name for Excel output

cols=['typeCode', 'freqCode', 'refPeriodId', 'refYear', 'period',
       'reporterCode','reporterDesc', 'flowCode', 'flowDesc',
       'partnerCode', 'partnerDesc', 'partner2Code',
       'partner2Desc', 'classificationCode',
       'classificationSearchCode', 'isOriginalClassification', 'cmdCode',
       'cmdDesc', 'aggrLevel', 'isLeaf', 'customsCode', 'customsDesc',
       'mosCode', 'motCode', 'motDesc', 'qtyUnitCode', 'qtyUnitAbbr', 'qty',
       'isQtyEstimated', 'altQtyUnitCode', 'altQtyUnitAbbr', 'altQty',
       'isAltQtyEstimated', 'netWgt', 'isNetWgtEstimated', 'grossWgt',
       'isGrossWgtEstimated', 'cifvalue', 'fobvalue', 'primaryValue',
       'legacyEstimationFlag', 'isReported', 'isAggregate', 'cmdCodeAG2']

real_period = detail_commodities['refYear'].unique()
start_year=min(real_period)
end_year=max(real_period)
period=f"{start_year}-{end_year}"

filename_note=f"{period}_{flowCode.replace(',','_')}"  # change to append to filename
excel_file_name = f"./reports/{reporterDesc}_{partnerDesc}_cmd_detail_{filename_note}.xlsx"
excel_file = pd.ExcelWriter(excel_file_name)
# Save data
sheet_name = "comtrade"
detail_commodities_no_ndx = detail_commodities[cols].reset_index()
detail_commodities_no_ndx.to_excel(excel_file, sheet_name=sheet_name, index=True, startrow=2)

# Write title in first row TBD
excel_file_title = f"Commodities {reporterDesc}-{partnerDesc}, detail of top {rank_filter} (M=Import, X=Export), {period} USD"
excel_file.sheets[sheet_name].write(0, 0, excel_file_title)
print(excel_file_title)

# Adjust column width
comtradetools.excel_col_autowidth(detail_commodities_no_ndx, excel_file,sheet_name,consider_headers=False)

# format column primaryValue as currency
comtradetools.excel_format_currency(detail_commodities_no_ndx, excel_file,sheet_name, 
                                    columns=['primaryValue' ,'cifvalue', 'fobvalue',],width=20)

excel_file.close()
print("Saved to:",excel_file_name)


Commodities China-PLP, detail of top 5 (M=Import, X=Export), 2003-2022 USD
Saved to: ./reports/China_PLP_cmd_detail_2003-2022_M_X.xlsx


### Check details of a given partner

In [61]:
import comtradetools
# show more rows
pd.set_option('display.max_rows', 1000)
# select country
partner_of_interest = 'Brazil'

# select only leaf commodities (not aggregate upper levels)
detail_commodities[
    (detail_commodities['partnerDesc']==partner_of_interest)
    & (detail_commodities['isLeaf']==True)
    ]\
    [['reporterDesc','partnerDesc','flowCode','refYear','cmdCode','cmdDesc','isLeaf','partner2Code','partner2Desc','motCode','customsCode','isAggregate','primaryValue']]\
        .sort_values(by=['reporterDesc','partnerDesc','refYear','flowCode','primaryValue'],
                     ascending=[True,True,True,True, False]).head(30)

,reporterDesc,partnerDesc,flowCode,refYear,cmdCode,cmdDesc,isLeaf,partner2Code,partner2Desc,motCode,customsCode,isAggregate,primaryValue
1099,China,Brazil,M,2003,120100,Soya beans; whether or not broken,True,0,World,0,C00,False,"1,683,314,880.00"
1109,China,Brazil,M,2003,260111,Iron ores and concentrates; non-agglomerated,True,0,World,0,C00,False,"1,004,243,700.00"
1110,China,Brazil,M,2003,260112,Iron ores and concentrates; agglomerated (excluding roasted iron pyrites),True,0,World,0,C00,False,"343,389,832.00"
1134,China,Brazil,M,2003,720712,Iron or non-alloy steel; semi-finished products of iron or non-alloy steel; containing by weight...,True,0,World,0,C00,False,"194,086,411.00"
1152,China,Brazil,M,2003,720917,"Iron or non-alloy steel; in coils, flat-rolled, width 600mm or more, cold-rolled, of a thickness...",True,0,World,0,C00,False,"133,416,336.00"
1133,China,Brazil,M,2003,720711,Iron or non-alloy steel; semi-finished products of iron or non-alloy steel; containing by weight...,True,0,World,0,C00,False,"61,584,212.00"
1162,China,Brazil,M,2003,721049,"Iron or non-alloy steel; flat-rolled, width 600mm or more, (not corrugated), plated or coated wi...",True,0,World,0,C00,False,"54,794,875.00"
1143,China,Brazil,M,2003,720839,"Iron or non-alloy steel; in coils, without patterns in relief, flat-rolled, of a width 600mm or ...",True,0,World,0,C00,False,"50,051,333.00"
1160,China,Brazil,M,2003,721012,"Iron or non-alloy steel; flat-rolled, width 600mm or more, plated or coated with tin, thickness ...",True,0,World,0,C00,False,"29,669,698.00"
1128,China,Brazil,M,2003,720293,Ferro-alloys; ferro-niobium,True,0,World,0,C00,False,"24,450,720.00"


Temos que identificar as linhas com valor agregado porque
os dados não tem essa coluna correcta.

Na lista China-Angola importações a flag isAggregate parece não 
estar correcta até 2017 inclusive. 2018 já está bem.

See the [bug report](isaggregate_bug.ipynb)


In [19]:

df['valueForTotal'] = df['primaryValue']
df.loc[df['isAggregate'] == True,'valueForTotal'] = 0
df['subtotalAG2'] = df.groupby(['partnerCode','refYear','flowCode','cmdCodeAG2'])["valueForTotal"].transform('sum')
df['subtotalCmd'] = df.groupby(['partnerCode','refYear','flowCode','cmdCode'])["valueForTotal"].transform('sum')
df['percentAG2'] = df['primaryValue'] / df['subtotalAG2'] * 100
df['percentCmd'] = df['primaryValue'] / df['subtotalCmd'] * 100

In [20]:
import comtrade

excel_file_name = f"./reports/product_detail_{reporter_name}_{years_of_interest.replace(',','-')}_{flow}_{partner_name}.xlsx"
excel_file = pd.ExcelWriter(excel_file_name, engine='xlsxwriter')

df2 = df[pco_cols_detail+['percentAG2']].sort_values('primaryValue',ascending=False)
df2.to_excel(excel_file, sheet_name='detail', index=True, startrow=2)

comtrade.excel_col_autowidth(df2, excel_file, 'detail')
comtrade.excel_format_currency(df2, excel_file,'detail', columns=['primaryValue'],width=20)
excel_file.close()
print(f"Excel file {excel_file_name} created")


Excel file ./reports/product_detail_China_2021_M_Brazil.xlsx created


In [21]:

pd.options.display.max_rows = 500
# df.sort_values(['partnerDesc','refYear','flowCode','subtotalAG2','cmdCodeAG2','primaryValue'],ascending=[True,True,True,False,True,False])[pco_cols_detail].head(500)
# format columns percentAG2 to percentage 3 decimals
df['percentAG2Form'] = df['percentAG2'].map("{:.3f}%".format)
df['percentCmdForm'] = df['percentCmd'].map("{:.3f}%".format)
df[df['isAggregate'] == False].sort_values(['reporterDesc','partnerDesc','refYear','cmdCode','subtotalAG2','percentAG2'],ascending=[True,True,True,True,False,False])\
    [['reporterDesc','partnerDesc','refYear','partner2Desc','percentCmdForm','valueForTotal','cmdCodeAG2','cmdCode','cmdDesc']+\
    ['customsDesc','motDesc','subtotalCmd','subtotalAG2']].head(500)


,reporterDesc,partnerDesc,refYear,partner2Desc,percentCmdForm,valueForTotal,cmdCodeAG2,cmdCode,cmdDesc,customsDesc,motDesc,subtotalCmd,subtotalAG2
2,China,Brazil,2021,World,100.000%,"4,620,523,121.00",02,020230,"Meat; of bovine animals, boneless cuts, frozen",All customs procedure codes,All modes of transport,"4,620,523,121.00","7,799,680,540.00"
4,China,Brazil,2021,World,100.000%,"39,241,333.00",02,020322,"Meat; of swine, hams, shoulders and cuts thereof, with bone in, frozen",All customs procedure codes,All modes of transport,"39,241,333.00","7,799,680,540.00"
5,China,Brazil,2021,World,100.000%,"1,617,075,329.00",02,020329,"Meat; of swine, n.e.c. in item no. 0203.2, frozen",All customs procedure codes,All modes of transport,"1,617,075,329.00","7,799,680,540.00"
7,China,Brazil,2021,World,100.000%,"2,784,082.00",02,020500,"Meat; of horses, asses, mules or hinnies, fresh, chilled or frozen",All customs procedure codes,All modes of transport,"2,784,082.00","7,799,680,540.00"
9,China,Brazil,2021,World,100.000%,"61,929,222.00",02,020649,"Offal, edible; of swine, (other than livers), frozen",All customs procedure codes,All modes of transport,"61,929,222.00","7,799,680,540.00"
11,China,Brazil,2021,World,100.000%,"229,346.00",02,020712,"Meat and edible offal; of fowls of the species Gallus domesticus, not cut in pieces, frozen",All customs procedure codes,All modes of transport,"229,346.00","7,799,680,540.00"
12,China,Brazil,2021,World,100.000%,"1,455,270,569.00",02,020714,"Meat and edible offal; of fowls of the species Gallus domesticus, cuts and offal, frozen",All customs procedure codes,All modes of transport,"1,455,270,569.00","7,799,680,540.00"
14,China,Brazil,2021,World,100.000%,"2,627,538.00",02,020910,"Fat; pig fat, free of lean meat, not rendered or otherwise extracted, fresh, chilled, frozen, sa...",All customs procedure codes,All modes of transport,"2,627,538.00","7,799,680,540.00"
87,China,Brazil,2021,World,100.000%,522.00,12,120110,"Soya beans; seed, whether or not broken",All customs procedure codes,All modes of transport,522.00,"33,120,876,740.00"
88,China,Brazil,2021,World,100.000%,"33,120,436,899.00",12,120190,"Soya beans; other than seed, whether or not broken",All customs procedure codes,All modes of transport,"33,120,436,899.00","33,120,876,740.00"
